# Дообучение сегментатора на полных изображениях

In [1]:
from model import RootVolumeNet
from dataset import RootVolumeDataset
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [2]:
def nearest_power_of_2(n):
    return 2 ** int(np.ceil(np.log2(n)))

def find_max_slice(csv_path):
    df = pd.read_csv(csv_path)
    return nearest_power_of_2(np.max(df["End"] - df["Start"]) + 1)

def get_params():
    size_file = open("target_size.txt", "r")
    width, height = [int(line.replace("\n", "").split(" ")[-1]) for line in size_file.readlines()]
    params = {}
    params["width"] = width
    params["height"] = height
    params["max_slices"] = find_max_slice("Train.csv")
    return params

In [5]:
params = get_params()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_path = "Train.csv"
img_path = "images/train/"
dataset = RootVolumeDataset(csv_path=train_path,
                            img_root=img_path,
                            target_width=params['width'],
                            target_height=params['height'],
                            label_root="train_labels/",
                            device=device,
                            pre_segment=False)

In [6]:
print(dataset[0]["images"])

[<PIL.Image.Image image mode=RGB size=908x28 at 0x1E4F7867A60>, <PIL.Image.Image image mode=RGB size=908x28 at 0x1E4F7867550>, <PIL.Image.Image image mode=RGB size=908x28 at 0x1E4F7867610>, <PIL.Image.Image image mode=RGB size=908x28 at 0x1E4F7867640>, <PIL.Image.Image image mode=RGB size=908x28 at 0x1E4F78313F0>, <PIL.Image.Image image mode=RGB size=908x28 at 0x1E4F7831420>, <PIL.Image.Image image mode=RGB size=908x28 at 0x1E4F7831990>, <PIL.Image.Image image mode=RGB size=908x28 at 0x1E4F7831360>, <PIL.Image.Image image mode=RGB size=908x28 at 0x1E4F7831330>, <PIL.Image.Image image mode=RGB size=908x28 at 0x1E4F78312A0>]


In [12]:
imgs = dataset[60]["images"]
for image in imgs:
    display(image)